# 리눅스 인프라 구축을 통한 WordPress 구현

### 프로젝트 개요

- 학습 배경
    - 예전에는 온프레미스(On-premiss) 방식으로 네트워크나 하드웨어를 잘 아는 인프라 엔지니어가 담당했습니다.
최근에는 가상의 서버를 여러 대 띄우는 Cloud 방식으로 옮기게 되면서 개발자가 배포나 테스트 등을 직접 할 수 있게 되었고 이로 인해서 인프라 관련 지식이 필요하게 되었습니다.
기초적인 인프라 지식을 학습하며 , 인프라 구축을 진행하는 프로젝트를 진행해 보았습니다.


### 프로젝트 환경

- Apache
- Php
- MariaDB
- Word press

### 진행 과정

1. DB 서버 구축
    - MariaDB 설치 및 실행
        ```
        # dnf -y install mariadb-server

        # systemctl enable mariadb.service --now
        ```
    - MySQL 설정
        ```
        # mysql_secure_installation

        ===

        Enter current password for root (enter for none): 

        Switch to unix_socket authentication [Y/n] y
        Enabled successfully!
        Reloading privilege tables..
        ... Success!

        Change the root password? [Y/n] y
        New password: 
        Re-enter new password: 
        Password updated successfully!
        Reloading privilege tables..
        ... Success!

        Remove anonymous users? [Y/n] y
        ... Success!

        Disallow root login remotely? [Y/n] n
        ... skipping.

        Remove test database and access to it? [Y/n] n
        ... skipping.

        Reload privilege tables now? [Y/n] y
        ... Success!

        ===
        ```
    - DB 생성하기
        ```
        - MySql 접속

        # mysql -u root -p
        Enter password: 
        Welcome to the MariaDB monitor.  Commands end with ; or \g.
        Your MariaDB connection id is 8
        Server version: 10.5.22-MariaDB MariaDB Server

        Copyright (c) 2000, 2018, Oracle, MariaDB Corporation Ab and others.

        Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

        - DB 생성하기
        MariaDB [mysql]> CREATE DATABASE wpdb1;
        Query OK, 1 row affected (0.009 sec)

        MariaDB [mysql]> CREATE USER wpdbuser1@'%' IDENTIFIED BY '1';
        Query OK, 0 rows affected (0.010 sec)

        MariaDB [mysql]> GRANT ALL ON wpdq1.* TO wpdbuser1@'%';
        Query OK, 0 rows affected (0.010 sec)

        MariaDB [mysql]> FLUSH PRIVILEGES;
        Query OK, 0 rows affected (0.009 sec)

        MariaDB [(none)]> exit
        Bye
        ```

2. 웹 서버 구축
    - ip 부여하기
        ```
        # nmcli con add con-name staticweb type ethernet ifname eth1 ip4 192.168.56.100/24 gw4 192.168.56.1

        # nmcli con up staticweb

        # nmcli con show
        NAME         UUID                                  TYPE      DEVICE 
        eth0         5fb06bd0-0bb0-7ffb-45f1-d6edd65f3e03  ethernet  eth0   
        staticweb    be23afd6-ddf3-4004-ae5e-42ce0ea6984c  ethernet  eth1   
        enp0s3       2c12d263-f3e2-440c-8b4e-6ef4e60716ae  ethernet  --     
        System eth1  9c92fad9-6ecb-3e6c-eb4d-8a47c6f50c04  ethernet  --
        ```
    - Apache 설치
        ```
        # dnf -y install httpd

        # firewall-cmd --add-service=http --permanent
        # firewall-cmd --reload
        ```
    - php 설치
        ```
        # dnf module enable php:8.1

        # dnf -y install php
        ```
    - 관련 패키지 설치
        ```
        dnf install php-gd php-soap php-intl php-mysqlnd php-pdo php-pecl-zip php-fpm php-opcache php-curl php-zip wget
        ```
    - 파일 설정 변경
        ```
        # vim /etc/httpd/conf/httpd.conf

        168 <IfModule dir_module>
        169     DirectoryIndex index.html index.php
        170 </IfModule>

        288     AddType application/x-compress .Z
        289     AddType application/x-gzip .gz .tgz
        290     AddType application/x-httpd-php .php
        291     AddType application/x-https-phps .phps
        ```
        ```
        # vim /etc/php-fpm.d/www.conf

        55 listen.acl_users = apache,nginx
        56 listen.acl_groups = apache
        ```

        ```
        # vi /var/www/html/phpinfo.php

        <?php phpinfo();?>
        ```
    - 패키지 실행
        ```
        # systemctl restart httpd

        # systemctl enable php-fpm --now
        ```
    - wordpress 다운
        ```
        # wget https://wordpress.org/latest.tar.gz -O wordpress.tar.gz

        # tar xf wordpress.tar.gz -C /var/www/html
        ```

    - 권한 부여
        ```
        # chown -R apache:apache /var/www/html/wordpress/

        # chmod -R 755 /var/www/html/wordpress/
        ```

    - host파일 생성
        ```
        # cp /usr/share/doc/httpd-core/httpd-vhosts.conf /etc/httpd/conf.d/wordpress.conf
        ```

    - host파일 편집
        ```
        # vim /etc/httpd/conf.d/wordpress.conf

        <VirtualHost *:80>
            ServerAdmin root@test.example.com
            DocumentRoot "/var/www/html/wordpress"
            ServerName wp.test.example.com
            ServerAlias wp
            ErrorLog "/var/log/httpd/dummy-host.example.com-error_log"
            CustomLog "/var/log/httpd/dummy-host.example.com-access_log" common
        </VirtualHost>

        <Directory "/var/www/html/wordpress">
        Options Indexes FollowSymLinks
        AllowOverride all
        Require all granted
        </Directory>
        ```

    - 아파치 재실행
        ```
        # systemctl restart httpd
        ```

3. DNS 서버 구축
    - ip 설정
        ```
        # nmcli con add con-name static1 type ethernet ifname eth1 ip4 192.168.56.200/24 gw4 192.168.56.1

        # nmcli con up static1  
        ```
    - 네트워크 설정
        ```
        # vim /etc/resolv.conf
        ---
        # Generated by NetworkManager
        nameserver 192.168.56.200
        ```
    - 패키지 설치
        ```
        # dnf -y install bind
        ```
    - 패키지 실행
        ```
        # systemctl enable named.service --now
        # systemctl start firewalld.service 
        ```
    - 정방향 만들기
        ```
        디렉토리 이동
        cd /var/named/

        cp named.empty test.example.com.zone

        vim test.example.com.zone 

        ==========

        $TTL 3H
        @       IN SOA   test.example.com. root.test.example.com. (
                                                0       ; serial
                                                1D      ; refresh
                                                1H      ; retry
                                                1W      ; expire
                                                3H )    ; minimum
                NS      dns.test.example.com.
                A       192.168.56.200

        wp IN   A       192.168.56.100
        db IN   A       192.168.56.150
        dns IN  A       192.168.56.200
        ==========

        권한 부여
        chown :named test.example.com.zone 
        ```
    - 역방향 만들기
        ```
        cp test.example.com.zone 192.168.56.zone

        vim 192.168.56.zone 

        ==========

        $TTL 3H
        @       IN SOA   test.example.com. root.test.example.com. (
                                                0       ; serial
                                                1D      ; refresh
                                                1H      ; retry
                                                1W      ; expire
                                                3H )    ; minimum
                NS      dns.test.example.com.
                A       192.168.56.200

        100 IN  PTR     wp.test.example.com.
        150 IN  PTR     db.test.example.com.
        200 IN  PTR     dns.test.example.com.
        ==========

        권한 부여
        # chown :named 192.168.56.zone
        ```
    - 반영하기
        ```
        # systemctl restart named
        ```
    - dns 방화벽에 추가하기
        ```
        # firewall-cmd --add-service=dns --permanent 
        success

        # firewall-cmd --reload
        ```
    - 연결 확인해보기
        ```
        # nslookup wp.test.example.com 192.168.56.200
        ```

4. SSL
    - 패키지 설치
        ```
        # dnf -y install mod_ssl

        # firewall-cmd --add-service=https --permanent 
        # firewall-cmd --reload
        ```
    - 패키지 버전 체크
        ```
        # httpd -v
        Server version: Apache/2.4.53 (Rocky Linux)
        Server built:   Apr 28 2023 00:00:00

        # openssl version
        OpenSSL 3.0.1 14 Dec 2021 (Library: OpenSSL 3.0.1 14 Dec 2021)
        ```
    - 공개키, crt, csr
        ```
        # openssl genrsa -out private.key 2048
        # openssl req -new -key private.key -out cert.csr
        # openssl x509 -req -signkey private.key -in cert.csr -out cert.crt

        파일이동
        # mp private.key /etc/pki/tls/private/
        # mv cert.crt /etc/pki/tls/certs/
        ```
        ```
        vim /etc/httpd/conf.d/ssl.conf

        =========

        43 DocumentRoot "/var/www/html/wordpress"
        44 ServerName wp.test.example.com:443

        59 SSLProtocol all -SSLv3 -TLSv1 -TLSv1.1

        73 SSLCipherSuite ECDHE-ECDSA-AES128-GCM-SHA256:ECDHE-RSA-AES128-GCM-SHA256:ECDHE-ECDSA-AES256-GCM-SHA384:ECDHE-RSA-AES256-GCM-SHA384    :ECDHE-ECDSA-CHACHA20-POLY1305:ECDHE-RSA-CHACHA20-POLY1305:DHE-RSA-AES128-GCM-SHA256:DHE-RSA-AES256-GCM-SHA384:DHE-RSA-CHACHA20-PO    LY1305

        85 SSLCertificateFile /etc/pki/tls/certs/cert.crt

        93 SSLCertificateKeyFile /etc/pki/tls/private/private.key
        ```
        ```
        # systemctl restart httpd
        ```

https://ssl-config.mozilla.org/

참고사항
- 추후 다양한 가상 머신을 구축한다면 로드밸런싱을 활용하는 방안도 고려해볼 수 있습니다.